In [1]:
data = [line.strip().split('\t') for line in open('project3_dataset4.txt', 'r')]

In [2]:
import numpy as np
import math
from sklearn import preprocessing
data = np.asarray(data)
print(data)
classlabels = data[:,-1].reshape((data.shape[0],1)).astype(int)
data = data[:,0:-1]
print(classlabels)
print(data)



enc = preprocessing.OneHotEncoder()
enc.fit(data)
data = enc.transform(data).toarray()
print(data)

[['sunny' 'hot' 'high' 'weak' '0']
 ['sunny' 'hot' 'high' 'strong' '0']
 ['overcast' 'hot' 'high' 'weak' '1']
 ['rain' 'mild' 'high' 'weak' '1']
 ['rain' 'cool' 'normal' 'weak' '1']
 ['rain' 'cool' 'normal' 'strong' '0']
 ['overcast' 'cool' 'normal' 'strong' '1']
 ['sunny' 'mild' 'high' 'weak' '0']
 ['sunny' 'cool' 'normal' 'weak' '1']
 ['rain' 'mild' 'normal' 'weak' '1']
 ['sunny' 'mild' 'normal' 'strong' '1']
 ['overcast' 'mild' 'high' 'strong' '1']
 ['overcast' 'hot' 'normal' 'weak' '1']
 ['rain' 'mild' 'high' 'strong' '0']]
[[0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]]
[['sunny' 'hot' 'high' 'weak']
 ['sunny' 'hot' 'high' 'strong']
 ['overcast' 'hot' 'high' 'weak']
 ['rain' 'mild' 'high' 'weak']
 ['rain' 'cool' 'normal' 'weak']
 ['rain' 'cool' 'normal' 'strong']
 ['overcast' 'cool' 'normal' 'strong']
 ['sunny' 'mild' 'high' 'weak']
 ['sunny' 'cool' 'normal' 'weak']
 ['rain' 'mild' 'normal' 'weak']
 ['sunny' 'mild' 'normal' 'strong']
 ['overcast' 'mild' 'hig

In [3]:
def is_categorical(s):
    try:
        float(s)
    except ValueError:
        return True
    return False


def evaluate(actual, predicted):
    # a=TP, b= FN, c=FP, d= TN
    a = b = c = d = 0
    for i in range(len(actual)):
        if(actual[i] == 1 and predicted[i] == 1):
            a += 1
        elif(actual[i] == 1 and predicted[i] == 0):
            b += 1
        elif(actual[i] == 0 and predicted[i] == 1):
            c += 1
        elif(actual[i] == 0 and predicted[i] == 0):
            d += 1
    return a, b, c, d

In [4]:
# strindex = []
# for i in range(len(data[0])):
#     if(is_categorical(data[0][i])):
#         print(data[0][i])
#         print(i)
#         strindex.append(i)
# for i in strindex:
#     unique_vals = np.unique(data[:,i])
#     mapping = list(range(len(unique_vals)))
#     dictionary = dict(zip(unique_vals, mapping))
#     for j in range(len(data[:,i])):
#         data[j][i] = dictionary.get(data[j][i])

data = data.astype(float)
data = np.append(data,classlabels,axis=1)
print(data)

[[0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1.]
 [0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
 [0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0.]]


In [5]:
print(data.shape)

(14, 11)


In [6]:
def calculate_gini(left, right):
    prob_left0 = prob_left1 = prob_right0 = prob_right1 = 0
    l = len(left)
    r = len(right)
    total = l + r
    if(l != 0):
        prob_left0 = float(list(left[:,-1]).count(0)) / l
        prob_left1 = float(list(left[:,-1]).count(1)) / l
    if(r != 0):
        prob_right0 = float(list(right[:,-1]).count(0)) / r
        prob_right1 = float(list(right[:,-1]).count(1)) / r
    
    gini_left = 1.0 - ((prob_left0 ** 2) + (prob_left1 ** 2))
    gini_right = 1.0 - ((prob_right0 ** 2) + (prob_right1 ** 2))
    
    gini = float((gini_left * l) + (gini_right * r)) / total
    return gini


In [7]:
def split_data(data,j,splitval):
    left = []
    right = []
    for i in range(len(data)):
        if(data[i][j] < splitval):
            left.append(data[i])
        else:
            right.append(data[i])
    left = np.asarray(left)
    right = np.asarray(right)
    return left, right

In [15]:
def node_to_split(data):
    min = float('inf')
    node = dict()
    flag=0
    for j in range(len(data[0])-1):
        if(splitted[j]!=1):
            for i in range(len(data)):
                left, right = split_data(data,j,data[i][j])
                gini = calculate_gini(left, right)
                if(gini < min):
                    min = gini
                    node['left'] = left
                    node['right'] = right
                    node['gini'] = min
                    node['split_id'] = i
                    node['splitval'] = data[i][j]
#                     print("i:",i," j: ",j," dta[i][j]: ",data[i][j])
                    node['split_feat'] = j
                    splitj = j
                    flag=1
                    
    if(flag==1):
        splitted[splitj] = 1
#     print("i:",i," j: ",j," dta[i][j]: ",data[i][j])    
    return node


def node_label(left, right):
    num0s = 0
    num1s = 0;
    if(len(left) != 0):
        num0s += list(left[:,-1]).count(0)
        num1s += list(left[:,-1]).count(1)
    if(len(right) != 0):
        num0s += list(right[:,-1]).count(0)
        num1s += list(right[:,-1]).count(1)
    return 1 if(num1s > num0s) else 0


def is_sameclass(data):
    unique_labels = np.unique(data[:,-1])
    if(len(unique_labels) == 1):
        return True
    return False


In [10]:
def build_tree(node):
    left = node['left']
    right = node['right']
    del(node['left'])
    del(node['right'])

    if(len(left) == 0 or len(right) == 0):
        node['left'] = node['right'] = node_label(left, right)
        return node 
#     if((left.size==0) or (right.size==0)):
#         classes = list()
#         if(left.size==0):
#             classes = right[:,-1]  
#         else:
#             classes = left[:,-1]
            
#         node['left'] = node['right'] = max(np.unique(classes), key = list(classes).count)
#         return node

    if(len(left) > 0):
        if(is_sameclass(left)):
            node['left'] = node_label(left, [])
        else:
            node['left'] = build_tree(node_to_split(left))
    if(len(right) > 0):
        if(is_sameclass(right)):
            node['right'] = node_label([], right)
        else:
            node['right'] = build_tree(node_to_split(right))        
    return node

    

        


In [17]:
def print_tree(rootnode):
    print(rootnode['gini'])
    print(rootnode['split_id'])
    print(rootnode['split_feat'])
    print(rootnode['left'])
    print("=====================================================")
    print(rootnode['right'])

In [12]:
def classify(node, test_point):
#     if(test_point[node['split_feat']] < node['splitval']):
#         if(node['left'] == 0 or node['left'] == 1):
#             return node['left']                                 
#         else:
#             return classify(node['left'], test_point)
#     else:
#         if(node['right'] == 0 or node['right'] == 1):
#             return node['right']
#         else:
#             return classify(node['right'], test_point)

    if(test_point[node['split_feat']] < node['splitval']):
        if isinstance(node['left'], dict):
            return classify(node['left'], test_point)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return classify(node['right'], test_point)
        else:
            return node['right']
   

In [13]:
def split(kfoldlist,index):
    test = kfoldlist[index]
    train = np.vstack([j for i,j in enumerate(kfoldlist) if i != index])
    train = np.asarray(train)
    test = np.asarray(test)
    return train, test

In [16]:
# kfoldlist = np.array_split(data, 10)
# accuracy = precision = recall = f_measure = 0

train = data
splitted = [0]*(len(data[0])-1)
splitj = -1
rootnode = node_to_split(train)
rootnode = build_tree(rootnode)
print_tree(rootnode)

# for i in range(len(kfoldlist)):
#     train, test = split(kfoldlist,i)
#     test_labels = []
#     splitted = [0]*(len(data[0])-1)
#     splitj = -1
#     rootnode = node_to_split(train)
#     rootnode = build_tree(rootnode)
#     print_tree(rootnode)
#     for j in range(len(test)):
#         test_labels.append(classify(root,test[j]))
#     test_labels = np.asarray(test_labels)
#     a, b, c, d = evaluate(test[:,-1], test_labels)
#     accuracy += float(a + d)/(a + b + c + d)
# print("Accuracy: ",accuracy*10,"%")


0.35714285714285715
2
0
{'gini': 0.31999999999999984, 'split_id': 0, 'splitval': 1.0, 'split_feat': 6, 'left': {'gini': 0.2, 'split_id': 1, 'splitval': 1.0, 'split_feat': 8, 'left': 1, 'right': {'gini': 0.0, 'split_id': 0, 'splitval': 1.0, 'split_feat': 1, 'left': 1, 'right': 0}}, 'right': {'gini': 0.2, 'split_id': 0, 'splitval': 1.0, 'split_feat': 2, 'left': {'gini': 0.0, 'split_id': 0, 'splitval': 1.0, 'split_feat': 9, 'left': 0, 'right': 1}, 'right': 0}}
1
